In [56]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List,Tuple
from dataclasses import dataclass
from gt.solutions.br import BrownRobinsonOptimizer
from IPython.display import display

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{
  "shell_port": 51523,
  "iopub_port": 51524,
  "stdin_port": 51525,
  "control_port": 51526,
  "hb_port": 51527,
  "ip": "127.0.0.1",
  "key": "6d8abfdf-831d8cb6f79fe7a901721850",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-fcd1efde-7c03-4314-a8ca-dd750246db1c.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned ON


In [81]:
original_payoff_matrix = np.array([
    [19, 6, 8, 2, 7],
    [7, 9, 2, 0, 12],
    [3, 18, 11, 9, 10],
    [19, 10, 6, 19, 4],
])
test_payoff_matrix = np.array([
    [5, 8, 7, 5, 4],
    [1, 10, 5, 5, 6],
    [2, 4, 3, 6, 2],
    [3, 5, 4, 12, 3],
])
start_payoff_matrix = original_payoff_matrix

In [82]:
def to_df(matrix: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame(data=matrix,
                        index=[f'a{i+1}' for i in range(matrix.shape[0])],
                        columns=[f'b{i+1}' for i in range(matrix.shape[1])])
    
payoff_matrix = to_df(start_payoff_matrix)
payoff_matrix

,b1,b2,b3,b4,b5
a1,19,6,8,2,7
a2,7,9,2,0,12
a3,3,18,11,9,10
a4,19,10,6,19,4


In [87]:
def create_risk_table(matrix):
    matrix = matrix.copy()
    max_reward = matrix.max(axis=0)
    logger.debug(f"Max rewards for risk table:\n{max_reward}")
    for column in matrix.columns:
        matrix[column] = max_reward[column] - matrix[column]
    return matrix

print("RISK TABLE")
create_risk_table(payoff_matrix)

03:50:53 DEBUG:Max rewards for risk table:
b1    19
b2    18
b3    11
b4    19
b5    12
dtype: int32


RISK TABLE


,b1,b2,b3,b4,b5
a1,0,12,3,17,5
a2,12,9,9,19,0
a3,16,0,0,10,2
a4,0,8,5,0,8


In [84]:
def criteria(matrix: pd.DataFrame, hurwitz_alpha: float=0.5):
    criteria_matrix = pd.DataFrame(index=[f'a{i+1}' for i in range(matrix.shape[0])])
    criteria_matrix['criteria_bernoulli'] = matrix.mean(axis=1)
    criteria_matrix['criteria_wald'] = matrix.min(axis=1)
    criteria_matrix['criteria_optimistic'] = matrix.max(axis=1)
    criteria_matrix['criteria_hurwitz'] = hurwitz_alpha * criteria_matrix['criteria_wald'] + (1-hurwitz_alpha) * criteria_matrix['criteria_optimistic']
    # make a risk table for Savage criteria
    risk_table = create_risk_table(matrix)
    logger.info("PLEASE NOTE THAT SAVAGE CRITERIA IS NEGATED SO THAT max() could work next "
                "instead of min (as correct formulae here suggests: "
                "https://en.wikipedia.org/wiki/Regret_(decision_theory))")
    criteria_matrix['criteria_savage'] = -risk_table.max(axis=1)
    
    # count advisory score - number of times the strategy a_i is best in terms of criteria
    scores = np.zeros(len(matrix))
    criteria = [column for column in criteria_matrix.columns if column.startswith('criteria_')]
    for cri in criteria:
        logger.debug(f"Looking at criteria: {cri}")
        best_cri_value = criteria_matrix[cri].max()
        for i in range(len(criteria_matrix)):
            if criteria_matrix.iloc[i][cri] == best_cri_value:
                logger.info(f"Column {i} gets one point as its '{cri}' has best value of {best_cri_value}")
                scores[i] += 1
                
    criteria_matrix['scores_best'] = scores 
    return pd.concat([matrix, criteria_matrix], axis=1)
    
payoff_criteria_matrix = criteria(payoff_matrix)
print("CRITERIA MATRIX")
payoff_criteria_matrix

03:49:33 INFO:PLEASE NOTE THAT SAVAGE CRITERIA IS NEGATED SO THAT max() could work next instead of min (as correct formulae here suggests: https://en.wikipedia.org/wiki/Regret_(decision_theory))
03:49:33 DEBUG:Looking at criteria: criteria_bernoulli
03:49:33 INFO:Column 3 gets one point as its 'criteria_bernoulli' has best value of 11.6
03:49:33 DEBUG:Looking at criteria: criteria_wald
03:49:33 INFO:Column 3 gets one point as its 'criteria_wald' has best value of 4
03:49:33 DEBUG:Looking at criteria: criteria_optimistic
03:49:33 INFO:Column 0 gets one point as its 'criteria_optimistic' has best value of 19
03:49:33 INFO:Column 3 gets one point as its 'criteria_optimistic' has best value of 19
03:49:33 DEBUG:Looking at criteria: criteria_hurwitz
03:49:33 INFO:Column 3 gets one point as its 'criteria_hurwitz' has best value of 11.5
03:49:33 DEBUG:Looking at criteria: criteria_savage
03:49:33 INFO:Column 3 gets one point as its 'criteria_savage' has best value of -8


CRITERIA MATRIX


,b1,b2,b3,b4,b5,criteria_bernoulli,criteria_wald,criteria_optimistic,criteria_hurwitz,criteria_savage,scores_best
a1,19,6,8,2,7,8.4,2,19,10.5,-17,1.0
a2,7,9,2,0,12,6.0,0,12,6.0,-19,0.0
a3,3,18,11,9,10,10.2,3,18,10.5,-16,0.0
a4,19,10,6,19,4,11.6,4,19,11.5,-8,5.0


b1     5
b2    10
b3     7
b4    12
b5     6
dtype: int32